# **Project: Default of Credit Card Clients**

## <font color=darkgreen>*Members*: 
> ### <font color=blue>**Chandan Patel**<br>**Rahul Bankey** <br>**Chandni Kumari**<br>**Divyanshi Singh**

## Objective: 

### Dataset Description
    
### Methods and Steps followed in this notebook:
> - #### Step 1: Importing the Data
> - #### Step 2: Data Preprocessing
> - #### Step 3: Data Exploration using PySpark SQL
> - #### Step 4: Feature Engineering
> - #### Step 5: Machine Learning Models
    Prepare data for machine learning.<br>
    Split into training and testing data.<br>
    Create and evaluate Decision Trees model.<br>
    Create and evaluate Random Forest model.<br>
    Create and evaluate Naive Bayes model.<br>
    Create and evaluate Multilayer Perceptron Neural Network model.
> - #### Step 6: Clustering K-Means
    Clustering using KMeans
> - #### Step 7: Rest of the Machine Learning Models
    Additional machine learning models (Decision Trees, Random Forest, Naive Bayes, MLP)
> - #### Step 8: Model Evaluation
    Model evaluation (accuracy, precision, recall, F1)
> - #### Step 9: Interpreting the Best Classification
    Displaying and interpreting the best classification model<br>
    Displaying performance metrics<br>

In this notebook we will use the  dataset that provides information related to the credit cards clients and store it the spark datawarehouse. We will then use the datawarehouse to access this data using spark sql.


## Step 1: Importing the Data

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import (RandomForestClassifier, 
                                      DecisionTreeClassifier, 
                                      NaiveBayes,
                                      MultilayerPerceptronClassifier)
from pyspark.ml.clustering import KMeans
from pyspark.sql import Row

#### Initialize Spark Session

In [2]:
from pyspark.sql import SparkSession;
from pyspark.context import SparkContext;

# warehouse_location points to the default location for managed databases and tables
from os.path import abspath
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    . builder \
    .master("local[*]") \
    .appName("Big Data Analytics Project - Team Data Wizards") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

sc =spark.sparkContext
sc.setLogLevel("ERROR") # only display errors (not warnings)

# note: If you have multiple spark sessions running (like from a previous notebook you've run), 
# this spark session webUI will be on a different port than the default (4040). One way to 
# identify this part is with the following line. If there was only one spark session running, 
# this will be 4040. If it's higher, it means there are still other spark sesssions still running.
spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

23/11/04 21:51:57 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 10.21.19.148 instead (on interface eth0)
23/11/04 21:51:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/04 21:51:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session WebUI Port: 4040


In [3]:
spark

In [4]:
spark.catalog.listTables()

[]

In [5]:
## Show theexisting  databases 
df=spark.sql("show databases")
df.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



#### List the tables in the database.

In [6]:
tables = spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



Now, let's load the credit card dataset into the datawarehouse. We will use the spark dataframe API to load the data. We will then use the spark sql API to create a table from the dataframe. Load the file into a DataFrame and remove the header row.

In [11]:
df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("credit_card_clients.xls")
ccRaw.take(5)

Py4JJavaError: An error occurred while calling o71.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: com.crealytics.spark.excel. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessor